## notebook to load .ptseries parcels and export them as tsv for the feature visualizer

In [27]:
import nibabel as nb
import nilearn as nl
import hcp_utils as hcp
import numpy as np
import pandas as pd
import glob

## First average the files together

In [ ]:
#which files exist?
asc_flist = open("hbn_asc_50.txt",'r').read().splitlines()
nt_flist = open("hbn_nt_50.txt",'r').read().splitlines()

asc_exists=[]
nt_exists=[]

n_asc=0
n_nt=0
print('----- extracting ASCs -----')
for f in asc_flist:
    try:
        im_file = glob.glob(f'../sourcedata/data/HBN/brain/clean/{f}/ses-*/{f}_ses-*_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii')
        #print(im_file[0])
        img = nb.load(im_file[0])
#         img_y = img.get_fdata()
#         #img_y = scaler.fit_transform(img_y)
        asc_exists.append(im_file[0])
        print(f'exists  {f}')
        n_asc=n_asc+1
    except:
        print(f'file for {f} dne')
        
print('----- extracting NTs -----')
for f in nt_flist:
    try:
        im_file = glob.glob(f'../sourcedata/data/HBN/brain/clean/{f}/ses-*/{f}_ses-*_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii')
        #print(im_file[0])
        img = nb.load(im_file[0])
#         img_y = img.get_fdata()
#         #img_y = scaler.fit_transform(img_y)
        nt_exists.append(im_file[0])
        print(f'exists  {f}')
        n_nt=n_nt+1

    except:
        print(f'file for {f} dne')

In [68]:
#make the workbench commands to average all subjects to use in the terminal...
asc_command="wb_command -cifti-average ../sourcedata/data/HBN/brain/clean/sub-all/sub-asc_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii"
for asc in asc_exists:
    asc_command=asc_command+" -cifti "
    asc_command=asc_command+asc

nt_command="wb_command -cifti-average ../sourcedata/data/HBN/brain/clean/sub-all/sub-nt_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii"
for nt in nt_exists:
    nt_command=nt_command+" -cifti "
    nt_command=nt_command+nt


(750, 360)

In [75]:
for sub in ['nt', 'asc']:
    print(f'running {sub}')
    parcel_file=f'../sourcedata/data/HBN/brain/clean/sub-all/sub-{sub}_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii'
    img = nb.load(parcel_file)
    img_y = img.get_fdata()
    #img_y = scaler.fit_transform(img_y)

    #HBN TR is usually 800ms
    TR=0.8

    for i in np.arange(img_y.shape[1]):
        label=hcp.mmp['labels'][i+1] #odd that 0 is nothing '' ??? why?
        timecourse=img_y[:,i]
        onset=np.arange(timecourse.shape[0])*TR
        duration=np.ones(timecourse.shape[0])*TR
        data=np.column_stack((onset,duration,timecourse))
        #print(data)
        df=pd.DataFrame(data,columns=['onset','duration',label])
        df.to_csv(f'../outputs/parcel_timecourse_tsv/HBN/sub-all/{label}_{sub}.tsv', index=False, sep="\t")

running nt
running asc
